# Match Class

In [169]:
#| default_exp match_class

In [170]:
#| hide
from nbdev.showdoc import show_doc
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
from enum import Enum

### Overview
The following module defines the **Match class**, that will be used by the Matcher module later. This class provides a subview to the graph which we attempt to transform, and is created based on some match from the LHS pattern to a subgraph of the input graph (Further explanations on the meaning of matches are included in the next module).

This subview is set such that read-write operations can be done on nodes and edges imperatively, based on their symbolic name (the node/edge they match in the pattern) rather than their actual name in the input graph. That is, we can access nodes and edges in the input graph which were matched to the pattern (and only them), by using their corresponding names in the pattern. 

For each match found, the user of the library will recieve its correpsonding Match object, which can be used for imperative side effects.

### Requirements

In [171]:
#| export
import networkx as nx
from networkx import DiGraph
from typing import *
from graph_rewrite.core import _create_graph, draw, GraphRewriteException, NodeName, EdgeName

### Example
Assume that we have an input graph $G$:

In [172]:
draw(_create_graph(['A','B','C','D','E'], [('A','B'), ('A','C'),('D','E')]))

And a pattern which looks like:

In [173]:
draw(_create_graph(['1','2','3'], [('1','2'), ('1','3')]))

Intuitively, we can see that the pattern can be found in the input graph $G$, such that the pattern node $1$ corresponds to the graph node $A$, and the same goes for $2$ and $B$, $3$ and $C$. That mapping defines a match of the pattern in graph $G$ (We will dive into the definition of a match in the next module).

The corresponding Match object will allows us, for example, changing an attribute of node $A$ in the input graph, by accessing the symbolic name $1$ (which matches to $A$) and setting it to some dictionary of attributes. Note that the user will be able to access this class only after the transformation is done - therefore, if the transformation removed $A$ from the graph, the user won't be able to access the symbolic name $1$ anymore.

### Utils
Edge $(u,v)$ is represented in a Match with the name "u->v".

In [174]:
#| export
def convert_to_edge_name(src: NodeName, dest: NodeName) -> str:
    """Given a pair of node names, source and destination, return the name of the edge
    connecting the two in the format {src}->{dest}, which is the same format the parser
    uses to create edges in the pattern graph.

    Args:
        src (NodeName): A node name
        dest (NodeName): A node name

    Returns:
        str: A representative name for the edge (src, dest).
    """
    return f"{src}->{dest}"

A node is **anonymous** if its name begins with '_'. This notion allows us to define patters with anonymous nodes, whose existence we want to enforce, but which we do not use in the RHS part of the pattern and thus, can ignore them by not assigning any symbolic name to them.

In [175]:
#| export
def is_anonymous_node(node_name: NodeName) -> bool:
    """Given a name of a node in the pattern graph, return true if it begins with '$',
    which is the notion the parser uses to denote anonymous nodes.

    Args:
        node_name (NodeName): A node name in the pattern

    Returns:
        bool: Returns True if the node is anonymous, False otherwise.
    """
    return len(node_name) >= 1 and node_name[0] == '_'

### The Match Class

A **Match** is a subview of the original graph, limited to the nodes, edges and attributes specified in the pattern. It includes the original graph, dictionaries which map nodes/edges to their corresponding attributes in the pattern, and the mapping from pattern nodes to real ones.

In [176]:
#| export
class NodeType(Enum):
    """Enum of the different types of nodes in the pattern graph.
    """
    SINGLE = 1
    COLLECTION = 2

class Match:
    """Represents a single match of a pattern inside an input graph.
     Provides a subview to a graph, limited to the nodes, edges and attributes specified in the pattern.
    """
    def __init__(self, graph: DiGraph, nodes: List[NodeName], edges: List[EdgeName], 
                 mapping: Dict[NodeName, Set[NodeName]], node_type_mapping: Dict[NodeName, NodeType]):
        self.graph: DiGraph = graph
        self._nodes: List[NodeName] = nodes
        self._edges: List[EdgeName] = edges
        
        # Node names and edges can represent either single nodes or collections of nodes, 
        # so for each node name is mapped to a set of input nodes:
        self.mapping: Dict[NodeName, Set[NodeName]] = {k: {v} for k,v in mapping.items()}
        self.node_type_mapping: Dict[NodeName, NodeType] = NodeType
    
    # A function that checks if the node is valid and raises an exception if it is not
    def check_node_in_pattern(self, pattern_node: NodeName):
        if not pattern_node in self._nodes:
            raise GraphRewriteException(f"Node {pattern_node} does not exist in the pattern")
        
    # A function that checks if the edge is valid and raises an exception if it is not
    def check_edge_in_pattern(self, pattern_src: NodeName, pattern_dst: NodeName):
        if not (pattern_src, pattern_dst) in self._edges:
            raise GraphRewriteException(f"Edge {(pattern_src, pattern_dst)} does not exist in the pattern")
        
             
    # Two boolean functions that checks if the pattern node is a single node or a collection of nodes
    def is_collection(self, pattern_node: NodeName) -> bool:
        return self.node_type_mapping[pattern_node] == NodeType.COLLECTION

    def is_single(self, pattern_node: NodeName) -> bool:
        return self.node_type_mapping[pattern_node] == NodeType.SINGLE
    
    # Returns the node or the collection of nodes mapped to the pattern node
    # If the node is a single node, we return the single node that is mapped to it, otherwise we return a set of nodes
    def __get_node(self, pattern_node):
        self.check_node_in_pattern(pattern_node)
        if self.is_single(pattern_node):
            return self.graph.nodes[list(self.mapping[pattern_node])[0]]
        else:
            return [self.graph.nodes[node] for node in self.mapping[pattern_node]]
    
    # Returns the edge or the collection of edges mapped to the pattern edge
    # If the edge is between two single nodes, we return the edge itself, otherwise we return a set of edges
    def __get_edge(self, pattern_src, pattern_dst):
        self.check_edge_in_pattern(pattern_src, pattern_dst)
        src = self.mapping[pattern_src]
        dst = self.mapping[pattern_dst]
        if self.is_single(pattern_src) and self.is_single(pattern_dst):
            return self.graph.edges[src, dst]
        else:
            return [self.graph.edges[src, dst] for src in src for dst in dst]
    
    def nodes(self):
        return {pattern_node: self.__get_node(pattern_node) for pattern_node in self._nodes}        
    
    def edges(self):
        return {convert_to_edge_name(pattern_src, pattern_dest): self.__get_edge(pattern_src, pattern_dest) for (pattern_src, pattern_dest) in self._edges}

    def set_graph(self, graph: DiGraph):
        self.graph = graph

    def __eq__(self, other):
        if type(other) is Match and len(other.mapping.items()) == len(self.mapping.items()):
            return all([other.mapping.get(k) == v for k,v in self.mapping.items()])
        return False

    def __getitem__(self, key: Union[NodeName, str]):
        """Returns the node / edge of the input graph, which was mapped by the key in the pattern during matching.

        Args:
            key (Union[NodeName, str]): A symbolic name used by the pattern (for a node / edge)

        Raises:
            GraphRewriteException: If the key doesn't exist in the pattern, or is mapped to a node / edge
            which does not exist anymore (due to removal by the transformation, for example).

        Returns:
            The corresponding node / edge of the input graph
        """
        try:
            if str(key).__contains__("->") and len(str(key).split("->")) == 2:
                end_nodes = str(key).split("->")
                return self.__get_edge(end_nodes[0], end_nodes[1])
            else:
                return self.__get_node(key)
        except:
            raise GraphRewriteException(f"The symbol {key} does not exist in the pattern, or it was removed from the graph")
        
    def __str__(self):
        return self.mapping.__str__()

### Create a Match
This function will be used in the Matcher module, in order to convert a mapping from the pattern graph to the input graph into a corresponding instance of the Match class.

In [177]:
#| export
def mapping_to_match(input: DiGraph, pattern: DiGraph, collections_pattern: DiGraph, mapping: Dict[NodeName, Set[NodeName]],
                      node_type_mapping: Dict[NodeName,NodeType], filter: bool=True) -> Match:
    """Given a mapping, which denotes a match of the pattern in the input graph,
    create a corresponding instance of the Match class.

    Args:
        input (DiGraph): An input graph
        pattern (DiGraph): A pattern graph
        mapping (Dict[NodeName, NodeName]): A mapping of nodes in the pattern to nodes in the input graph
        node_type_mapping (Dict[NodeName, NodeType]): A mapping of nodes in the pattern to their type (single or collection)
        filter (bool): If True, filter out anonymous nodes from the match

    Returns:
        Match: A corresponding instance of the Match class
    """
    nodes_list, edges_list = [], []
    cleared_mapping = mapping.copy()

    for pattern_node in mapping.keys():
        if filter and is_anonymous_node(pattern_node):
            cleared_mapping.pop(pattern_node)
            continue # as we don't want to include this node in the Match
        nodes_list.append(pattern_node)

    for (n1, n2) in pattern.edges:
        if filter and (is_anonymous_node(n1) or is_anonymous_node(n2)):
            continue # as before
        edges_list.append((n1, n2))

    return Match(input, nodes_list, edges_list, cleared_mapping, node_type_mapping)

### Tests
We take the example graph and pattern described above, add a few attributes to the nodes and edges, and create a Match instance based on the single mapping:

In [178]:
G = _create_graph(
    [('A', {'name': 'A'}),('B', {'name': 'B'}),('C', {'name': 'C'}),('D', {'name': 'D'}),('E', {'name': 'E'})],
    [('A','B', {'edge_attr': 10}), ('A','C'),('D','E')])
pattern = _create_graph(['1','2','3'], [('1','2'), ('1','3')])
# It has a single match, which is defined by the following mapping:
mapping = {'1': 'A', '2': 'B', '3': 'C'}
mapping_match = mapping_to_match(G, pattern, DiGraph(), mapping, {})

In graph $G$, each node has a single attribute - "name" - whose value is the node's name. Therefore, node $A$ has the attribute "name" with the value "A". We can access its attributes using the Match, by accessing the name of the corresponding matched node $1$:

In [179]:
assert mapping_match['1'] == {'name': 'A'}
assert mapping_match['1']['name'] == 'A'
mapping_match['1']

{'name': 'A'}

This indicates that as expected, pattern node $1$ is matched in this Match instance to the input graph node $A$, as that node is the only one in the graph whose "name" attribute is equal to "A".

We can access the edges similarly, by using the format {src}->{dst} for edge $(src, dst)$:

In [180]:
assert mapping_match['1->2'] == {'edge_attr': 10}
mapping_match['1->2'] # accesses the attributes of the corresponding edge A->B in G

{'edge_attr': 10}

Say we want to add an attribute to $A$, we can change it in the same way using the Match:

In [181]:
mapping_match['1']['attr'] = 5
assert mapping_match['1'] == {'name': 'A', 'attr': 5}
mapping_match['1']

{'name': 'A', 'attr': 5}

And then see that the attributes of $A$ in the original graph $G$ have changed, as the Match is a subview of that graph, and so changes in the subview are reflected in $G$:

In [182]:
assert G.nodes(data=True)['A'] == {'name': 'A', 'attr': 5}
G.nodes(data=True)['A']

{'name': 'A', 'attr': 5}

We can also modify existing attributes in the same way using the Match, and see those changes reflected in $G$

In [183]:
mapping_match['2']['name'] = 'B*'
assert mapping_match['2']['name'] == 'B*'
assert G.nodes(data=True)['B']['name'] == 'B*'
G.nodes(data=True)['B']

{'name': 'B*'}

This also works in the reverse direction: changes in $G$ are reflected in the subview. Say that we set an attribute for edge $(A,B)$ in $G$, then that change would be reflected by accessing the edge '1->2' in the Match object that refers graph $G$:

In [184]:
G.edges()[('A','B')]['attr2'] = 20
assert mapping_match['1->2']['attr2'] == 20
mapping_match['1->2']

{'edge_attr': 10, 'attr2': 20}

## Visualizing matches

In [185]:
list(G.nodes(data=True)),list(G.edges(data=True))

([('A', {'name': 'A', 'attr': 5}),
  ('B', {'name': 'B*'}),
  ('C', {'name': 'C'}),
  ('D', {'name': 'D'}),
  ('E', {'name': 'E'})],
 [('A', 'B', {'edge_attr': 10, 'attr2': 20}), ('A', 'C', {}), ('D', 'E', {})])

In [186]:
mapping_match.nodes(),mapping_match._edges

({'1': {'name': 'A', 'attr': 5}, '2': {'name': 'B*'}, '3': {'name': 'C'}},
 [('1', '2'), ('1', '3')])

In [187]:
mapping_match.mapping

{'1': 'A', '2': 'B', '3': 'C'}

In [188]:
#| export
# Old, for reference
def draw_match(g,m,**kwargs):
    g_copy = g.copy()
    node_styles={}
    edge_styles={}
    for name_in_m,name_in_g in m.mapping.items():
        g_copy.nodes[name_in_g]['label']=name_in_m
        node_styles[name_in_g] = 'stroke:red,stroke-width:4px;'
    for u,v in m._edges:
        edge_styles[m.mapping[u],m.mapping[v]]='stroke:red,stroke-width:4px;'
    for name_in_m, collection in m.collection_mapping.items():
        for i, name_in_g in enumerate(collection):
            g_copy.nodes[name_in_g]['label']= (name_in_m + "_" + str(i))
            node_styles[name_in_g] = 'stroke:red,stroke-width:4px;'  

    draw(g_copy,node_styles=node_styles,edge_styles=edge_styles,**kwargs)

In [215]:
draw_match(G,mapping_match)

In [217]:
# Updated version
# We want to color the nodes and edges that are part of the match in blue, and specify the name of the pattern node
# Also, if more than one pattern node is mapped to the same input node, we want to color them in red, and specify the names of the pattern nodes
# and also notify the user (as a warning/notification) 
# To do that, we need to use a reverse mapping, where the key is the input node and the value is the pattern node
# TODO: make sure the semantics support this and that it makes sense mathematically 
def draw_match_v2(g, m, **kwargs):
    g_copy = g.copy()
    node_styles = {}
    edge_styles = {}
    reverse_mapping = {input_node: pattern_node for pattern_node, input_nodes in m.mapping.items() for input_node in input_nodes}
    for input_node, pattern_node in reverse_mapping.items():
        if len(reverse_mapping[input_node]) > 1:
            node_styles[input_node] = 'stroke:red,stroke-width:4px;'
            g_copy.nodes[input_node]['label'] = f"{pattern_node}_{input_node}"
            print(f"Warning: Node {input_node} is mapped to multiple pattern nodes: {reverse_mapping[input_node]}")
        else:
            node_styles[input_node] = 'stroke:blue,stroke-width:4px;'
            g_copy.nodes[input_node]['label'] = pattern_node
    for u, v in m._edges:
        edge_styles[m.mapping[u], m.mapping[v]] = 'stroke:blue,stroke-width:4px;'
    draw(g_copy, node_styles=node_styles, edge_styles=edge_styles, **kwargs)

Drawing the regular pattern in red:


Drawing the collections in blue:


# Export

In [218]:
#|hide
import nbdev; nbdev.nbdev_export()

: 